<a href="https://colab.research.google.com/github/GUNDESANDEEP/AWS-Lambda-inOne-Time-Encrypted-Link-Generator/blob/main/One_Time_Encrypted_Link_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Step 1: Import Required Modules
import uuid
import time
from cryptography.fernet import Fernet
from typing import Dict, Optional

In [4]:
# Step 2: Initialize Encryption System
print("🔐 Generating secure encryption key...")
encryption_key = Fernet.generate_key()
cipher = Fernet(encryption_key)

🔐 Generating secure encryption key...


In [5]:
# Step 3: In-memory "database" to simulate DynamoDB
secure_db: Dict[str, Dict] = {}

In [6]:
# Step 4: Secure Link Creation
def create_secure_link(message: str, expiry_seconds: int = 300) -> str:
    encrypted = cipher.encrypt(message.encode())
    link_id = str(uuid.uuid4())

    secure_db[link_id] = {
        "encrypted_data": encrypted,
        "used": False,
        "created_at": time.time(),
        "expiry": expiry_seconds
    }

    fake_url = f"https://securelink.fake/{link_id}"
    return fake_url

In [7]:
# Step 5: Secure Link Access
def access_secure_link(link_id: str) -> str:
    entry = secure_db.get(link_id)

    if entry is None:
        return "❌ Error: Invalid or expired link."

    if entry["used"]:
        return "⚠️ This link has already been used and cannot be accessed again."

    current_time = time.time()
    time_elapsed = current_time - entry["created_at"]

    if time_elapsed > entry["expiry"]:
        del secure_db[link_id]
        return "⏰ This link has expired."

    # Decrypt and return message
    decrypted = cipher.decrypt(entry["encrypted_data"]).decode()
    entry["used"] = True
    return f"✅ Decrypted Message: {decrypted}"

In [8]:
# Step 6: Admin View (For Debugging)
def list_all_links():
    print("\n📂 Secure Link Database (Admin View):")
    for link_id, info in secure_db.items():
        print(f"- ID: {link_id}")
        print(f"  Used: {info['used']}")
        print(f"  Expires In: {max(0, int(info['expiry'] - (time.time() - info['created_at'])))} seconds")
        print()

In [9]:
print("\n✅ Demo: Creating a secure one-time link...")
link = create_secure_link("Top Secret: Hackathon access code = X92-XYZ", expiry_seconds=120)
print("🔗 Generated Link:", link)

link_id = link.split("/")[-1]

# Accessing first time
print("\n➡️ First access:")
print(access_secure_link(link_id))

# Trying second access
print("\n❌ Second access:")
print(access_secure_link(link_id))

# Check DB Status
list_all_links()

# -------------------------------
# DEMO 2: Expiry Simulation
# -------------------------------
print("\n✅ Demo 2: Simulating expiry...")

# New Link with short expiry
short_link = create_secure_link("Self-destruct message: Goodbye!", expiry_seconds=3)
short_link_id = short_link.split("/")[-1]
print("🔗 Short-Lived Link:", short_link)

print("\n⏳ Waiting for 5 seconds...")
time.sleep(5)

print("➡️ Access after expiry:")
print(access_secure_link(short_link_id))


✅ Demo: Creating a secure one-time link...
🔗 Generated Link: https://securelink.fake/847d3574-b22b-4d9e-b9a4-8a51a097cd6c

➡️ First access:
✅ Decrypted Message: Top Secret: Hackathon access code = X92-XYZ

❌ Second access:
⚠️ This link has already been used and cannot be accessed again.

📂 Secure Link Database (Admin View):
- ID: 847d3574-b22b-4d9e-b9a4-8a51a097cd6c
  Used: True
  Expires In: 119 seconds


✅ Demo 2: Simulating expiry...
🔗 Short-Lived Link: https://securelink.fake/9ffde83e-34ff-464e-8694-4d2509163458

⏳ Waiting for 5 seconds...
➡️ Access after expiry:
⏰ This link has expired.
